# CS-660 Semester Project Notebook 1

## Stan Rosenbaum

Fall 2017

Anaconda 5 / Python 3

Using Keras with TensorFlow as back end.


First we init stuff.

In [1]:
# Load the Basic Python Libraries
import os
import csv
import PIL
import pickle
import random
import datetime

# Load my Data Management Module
import CS660DataManagement as csDM

# load numpy
import numpy as np

# Load Keras Studd
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform

from keras.callbacks import TensorBoard

from keras.utils.np_utils import to_categorical



# Other
import pydot
from IPython.display import SVG


# Not sure what this is:
# from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

# Get Processed Data Directory.
processedDataDir = os.path.join( os.getcwd(), os.pardir, 'DATA', 'PROCESSED' )
combinedDataDir = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED' )
pickleDataDir = os.path.join( os.getcwd(), os.pardir, 'DATA', 'PICKLES' )

testImageColorFile = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED', 'ICOLOR', 'TEST.png' )
testImageDepthFile = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED', 'IDEPTH', 'TEST.png' )
testCSVFile = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED', 'PCLOUD', 'TEST.csv' )

combinedDataDir = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED' )

imageDirs = ['ICOLOR', 'IDEPTH']
csvDirs = ['PCLOUD']

allDataFlavors = imageDirs + csvDirs

#Load main data file.
searCSVInfoFile = os.path.join( combinedDataDir, 'SEAR_DC_INFO.csv' )

csDM.CS660DataManagementCheck()

Using TensorFlow backend.


CS660DataManagementCheck Imported


## Generate the Learning and Verification sets

In [ ]:
print( len( csDM.getListOfDataCSVFileKeys() ) )
csDM.reportStats()
#csDM.createLearningAndVerificationPickles()


### Load a Training Set.
#### (Basing this all on the Coursera Course Code)



In [ ]:
def reshape(trainingSet):
    """
    This is to add the 'Channels' Dimension for Keras.
    """
    getShape = trainingSet.shape
    newShape = getShape + (1,)
    return trainingSet.reshape(newShape)
    

X_train, Y_train, X_test, Y_test = csDM.load_dataset( '1', 'ICOLOR' )



print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))

X_train = reshape(X_train)
X_test = reshape(X_test)

y_train_binary = to_categorical(Y_train, 4)
y_test_binary = to_categorical(Y_test, 4)

print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("y_train_binary shape: " + str(y_train_binary.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))
print ("y_test_binary shape: " + str(y_test_binary.shape))



## 2 - Building a model in Keras



In [ ]:
csDM.CS660DataManagementCheck()

In [28]:
import keras
from quiver_engine import server

# input image dimensions
img_rows, img_cols = 480, 640

num_classes = 4

x_train, y_train, x_test, y_test = csDM.load_dataset( '1', 'ICOLOR' )

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train_as_category = to_categorical(y_train, num_classes)
y_test_as_category = to_categorical(y_test, num_classes)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print('x_train shape:', x_train.shape)
print(y_train_as_category.shape, 'y_train shape')
print(y_test_as_category.shape, 'y_test shape')



def buildModel():

    model = Sequential()

    model.add(
        Conv2D(5, 
               kernel_size=(5, 5),
               strides=3,
               activation='relu',
               input_shape=input_shape
              )
    )
    model.add(
        MaxPooling2D(
            pool_size=(3, 3)
        )
    )

    model.add(
        Conv2D(
            10, 
            kernel_size=(3, 3),
            strides=1,
            activation='relu')
    )
    model.add(
        MaxPooling2D(
            pool_size=(2, 2)
        )
    )


    model.add(Flatten())

    for index in range(4):
        model.add(Dense(512))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(0.25))


    model.add(Dense(num_classes, activation='softmax'))

    model.compile(
        loss=keras.losses.categorical_crossentropy,
        optimizer=keras.optimizers.Adam(),
        metrics=['categorical_accuracy']
    )

    model.summary()

    return model
    
# server.launch(model)

model = buildModel()


x_train shape: (100, 480, 640, 1)
100 train samples
20 test samples
x_train shape: (100, 480, 640, 1)
(100, 4) y_train shape
(20, 4) y_test shape
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 159, 212, 5)       130       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 53, 70, 5)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 51, 68, 10)        460       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 25, 34, 10)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 8500)              0         
_________________________________________________________________
dense_41 (Dense)             (None, 512)               4352512

In [29]:
TBoardCallback = keras.callbacks.TensorBoard(
    log_dir='../TENSOR_LOGS/run_test', 
    histogram_freq=0,
    write_graph=True,
    write_images=True
)

model.fit(x_train,
          y_train_as_category,
          batch_size=16,
          epochs=32,
          verbose=1,
          validation_data=(x_test, y_test_as_category),
          callbacks = [TBoardCallback]
         )


Train on 100 samples, validate on 20 samples
Epoch 1/32
100/100 [==============================] - 1s - loss: 1.5077 - categorical_accuracy: 0.4200 - val_loss: 2.2786 - val_categorical_accuracy: 0.2500
Epoch 2/32
100/100 [==============================] - 1s - loss: 0.9731 - categorical_accuracy: 0.5300 - val_loss: 3.4030 - val_categorical_accuracy: 0.2500
Epoch 3/32
100/100 [==============================] - 1s - loss: 0.8746 - categorical_accuracy: 0.6800 - val_loss: 6.3880 - val_categorical_accuracy: 0.2500
Epoch 4/32
100/100 [==============================] - 1s - loss: 0.8111 - categorical_accuracy: 0.6200 - val_loss: 9.0293 - val_categorical_accuracy: 0.2500
Epoch 5/32
100/100 [==============================] - 1s - loss: 0.7847 - categorical_accuracy: 0.7100 - val_loss: 8.6053 - val_categorical_accuracy: 0.2500
Epoch 6/32
100/100 [==============================] - 1s - loss: 0.5981 - categorical_accuracy: 0.8000 - val_loss: 9.5201 - val_categorical_accuracy: 0.2500
Epoch 7/32
10

In [30]:
score = model.evaluate(x_test, y_test_as_category, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# serialize model to JSON
model_json = model.to_json()
with open("../MODELS/modelTEST.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../WEIGHTS/modelTEST.h5")
print("Saved model to disk")

Test loss: 1.80631220341
Test accuracy: 0.5
Saved model to disk


In [31]:
# Predict
print(x_test.shape)
results = model.predict_classes(x_test, verbose=1)
print(results)
for index in range(len(x_test)):
    print('prediction of: ', results[index], " VS ", y_test[index])



(20, 480, 640, 1)
20/20 [==============================] - 0s
[3 1 1 1 1 1 1 1 1 0 1 3 3 3 3 3 1 0 1 1]
prediction of:  3  VS  [ 2.]
prediction of:  1  VS  [ 2.]
prediction of:  1  VS  [ 1.]
prediction of:  1  VS  [ 2.]
prediction of:  1  VS  [ 1.]
prediction of:  1  VS  [ 1.]
prediction of:  1  VS  [ 2.]
prediction of:  1  VS  [ 0.]
prediction of:  1  VS  [ 1.]
prediction of:  0  VS  [ 1.]
prediction of:  1  VS  [ 2.]
prediction of:  3  VS  [ 3.]
prediction of:  3  VS  [ 3.]
prediction of:  3  VS  [ 3.]
prediction of:  3  VS  [ 3.]
prediction of:  3  VS  [ 3.]
prediction of:  1  VS  [ 0.]
prediction of:  0  VS  [ 0.]
prediction of:  1  VS  [ 0.]
prediction of:  1  VS  [ 0.]


In [57]:
plot_model(model, to_file='ModelTEST.png')
# SVG(model_to_dot(model).create(prog='dot', format='svg'))
print("")

Saved model to disk
